## Using Jupyter Widgets

Ok, we're towards the end of the course, and I want to introduce you to one more thing - Jupyterlab Widgets. Before I do
that though, let's reflect a bit on what you've learned. The first week of the course I showed you some methods from
software engineering -- specifically diagramming through class and interaction diagrams -- which I use to design
software solutions. These methods are commonly used in large teams, and are helpful in thinking through the entities and
relationships you want to represent in your software. Through this module we also practiced our object oriented
programming skills in python, and you tackled the creation of our game playing bot, WordyPy, in the assignment.

In the second week of the course we expanded beyond the python standard library, and learnt how to manipulate images
using the python imaging library. All third party libraries are different, so we went a bit in depth here, covering both
imaging fundamentals as well as the nuance of the PIL library itself. At the end of the week you tackled bringing image
manipulation into the WordyPy game to detect the state of the game.

Finally, in this week I introduced to you a new library, **tesseract**, which does optical character recognition. I
showed how this library works both by itself and with the skills you already have with the python imaging library. In
the assignment for this week you'll utilize tesseract in order to add state to the WordyPy game. Specifically, my game
engine will send you partial game plays which it expects you to finish, and you need to detect whatever the current game
state is in order to do that.

But before we do all of that, I want to introduce you to a form of interactivity called Jupyter Widgets.


### Jupyter Widgets

You've seen the real power first hand of the Jupyterlab notebooks -- quick editing that allows you to instantly see some
results in an easy to use web system. However, if you played at all with the extra images I gave you at the end of the
previous lecture then you have undoubtedly recognized that it can still be a bit of a pain to explore all of the
different options you might want to try when working with images. Especially when coming up with bounding boxes for
images, I find that this can be very slow to do in jupyter notebooks. What Jupyter widgets allow you to do, is move the
parameters for your functions out of your code and into the regular display functionality that you expect in a modern
computer. Really, the widgets allow you to build a very basic user interface to the objects and functions you've
written.

For this lecture, we're actually going to use a real problem I face! You see, I've been doing some research with
wearable devices, a platform called the biostrap which is a bit like a smart watch. This strap analyzes a few
physiological processes while you sleep, and provides a PDF report when you wake up. I actually have 50 of these I want
to analyze, all from different subjects, so a PDF is a bit of a pain to work with. Instead, I'm going to convert each of
these reports into images, and then use PIL and tesseract to extract all of the information I'm interested in.


In [ ]:
# First let's check out what this report looks like, here's one of mine
from PIL import Image, ImageDraw

image = Image.open("biostrap.png")
print(image)
display(image.resize((image.width // 4, image.height // 4)))

A few things to note. First, the image is very large -- I scaled it down to 25% size just to render it in the notebook.
It's ok that it's large though, this is good for OCR.

Next, we see there is lots of text. I actually don't want most of it, I just want the summary text in the top four
boxes, as this provides measures of the physiological processes I'm trying to understand. So we're going to do some
cropping here.

Finally, check out the mode of the file, it's RGBA. We know RGB stands for red, green, and blue color channels, but what
is the A color channel? Well, this is actually called the "alpha channel", and it captures transparency of a pixel. This
channel determines how the display can determine whether the pixel color should be shown, or should only be partially
shown depending on what's behind it on the screen. It's really quite cool, if we turn this value all the way up then the
image becomes fully transparent (invisible), while if we turn it all the way down it becomes just like any other image
we might want to show. If you've ever used dark mode for an editor or application and some of the images were completely
unreadable it's likely that they had a transparent background -- that is, the pixels that make up the background had RGB
values but also this alpha channel value -- and the designer for the website or application just assumed you would be
viewing it with a white background. In this example we're not going to use the alpha channel because we're going to
convert everything into greyscale for processing.


In [ ]:
# Let's see how we can use widgets for interaction. First, I'm going to make a small
# image that we'll work from which is 1/4 the size of the normal image
small_image = image.resize((image.width // 4, image.height // 4))

# Now I'm going to import the interact decorator from ipywidgets. This is not the only
# way to defined a user interface in Jupyter (the widgets), but it's the most straightforward
from ipywidgets import interact
import pytesseract


# The way this works is that we put the interact decorator right before the function we
# want to make interactive, and we ensure that the interact call matches the parameters
# of our function.


# For this example we have four parameters, so four on screen widgets, for the user.
# The first will be an integer, and it's bounded by 0 and the width of the image.
# The second is the same thing, and it's bounded by 0 and the height of the image.
# Together, these two parameters will let us specify the top left corner of the rectangle.
# Next we have a width and height, which I'll just set to 100. These are unbounded, and
# Jupyter is going to turn all of these into slides by default, and we can change that
# if we want to.
@interact(
    hr_left=(0, small_image.width),
    hr_top=(0, small_image.height),
    width=100,
    height=100,
)

# Now we just write the function we want to interact with. It must have the same
# parameters as the interact decorator, but unlike the decorator it actually does
# some work! When the user changes the values of the sliders, the decorator will
# call this function to rerender the image. I'm just going to render the small
# image with a red rectangle on it depending upon the parameters.
def draw(hr_left=0, hr_top=0, width=100, height=100):
    img = small_image.copy()
    drawing_object = ImageDraw.Draw(img)
    drawing_object.rectangle(
        (hr_left, hr_top, hr_left + width, hr_top + height), fill=None, outline="red"
    )
    display(img)

Now, this is pretty neat -- a quick way to build a little user interface right inside of Jupyter which is helpful for
finding parameters you might want to explore. Actually, there's a lot you can do with this, including video, date and
time pickers, string functions, and so forth. I won't go into this in detail, because it's a rapidly changing area and
out of scope for an introductory python course, but let me just walk you through what I did to tie this all together for
my analysis.

First, just observe that all of the boxes I'm interested in pulling data out of are in a row at the top of the image, so
each box will have the same top value and they all have the same width and height. This will make my application much
easier to write.


In [ ]:
# I'm going to work with the small image wherever possible
small_image = image.resize((image.width // 4, image.height // 4))

# And of course, we want to do some OCR here
import pytesseract


# I'm interested in four different measures, heart rate, blood oxygen, and two heart
# rate variability measures, and each one has its own offset from the left of the
# image. I also need a top coordinate which they all share, and a width and a height.
# I would like to make it optional to do the OCR, because that's slow, and because
# the quality of OCR gets better with full sized images, I would like the option to
# use the full sized image when I get the boxes lined up. Finally, I want to include
# a parameter for the tesseract options, which I'll set based on the previous lecture
@interact(
    hr_left=(0, small_image.width),
    spo2_left=(0, small_image.width),
    hrv1_left=(0, small_image.width),
    hrv2_left=(0, small_image.width),
    top=(0, small_image.height),
    width=100,
    height=100,
    do_ocr=False,
    ocr_full_size=False,
    ocr_tesseract_options='-c tessedit_char_whitelist=" -&.0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ\'"',
)

# Remember that the interact decorator is going to call this function whenever one of
# our widgets is changed, but this is the function that does the real work!
def draw(
    hr_left=0,
    spo2_left=0,
    hrv1_left=0,
    hrv2_left=0,
    width=100,
    top=0,
    height=100,
    do_ocr=False,
    ocr_full_size=False,
    ocr_tesseract_options="",
):
    # I've decided to nest the run_ocr function inside of the draw function. Remember
    # that this isn't run every time draw is called, it's just defined here and hidden
    # from the rest of the application. The OCR function is pretty much a copy and
    # paste from the previous lecture, I've just added a bit of logic so it will
    # change behavior based on whether the do_ocr flag is set.
    def run_ocr(image, left, top, width, height):
        # we just skip this all if they don't want to do OCR
        if do_ocr:
            # We use the small image unless they indicated full size
            ocr_image = small_image.copy()
            if ocr_full_size:
                ocr_image = image.copy()

            # Now we crop the image to the rectangle they specified
            if ocr_full_size:
                ocr_image = ocr_image.crop(
                    (left * 4, top * 4, left * 4 + width * 4, top * 4 + height * 4)
                )
            else:
                ocr_image = ocr_image.crop((left, top, left + width, top + height))

            # Now we'll run OCR on it and remove newlines
            return pytesseract.image_to_string(
                ocr_image, config=ocr_tesseract_options
            ).replace("\n", " ")
        return ""

    # Each time this function is called we will render an image. So we always want to start
    # with a "clean" copy of the image
    img = small_image.copy()
    drawing_object = ImageDraw.Draw(img)
    drawing_object.rectangle(
        (hr_left, top, hr_left + width, top + height), fill=None, outline="red"
    )
    drawing_object.rectangle(
        (spo2_left, top, spo2_left + width, top + height), fill=None, outline="blue"
    )
    drawing_object.rectangle(
        (hrv1_left, top, hrv1_left + width, top + height), fill=None, outline="green"
    )
    drawing_object.rectangle(
        (hrv2_left, top, hrv2_left + width, top + height), fill=None, outline="black"
    )

    # I'm going to print out the OCR results just with print(), which means they will
    # show up before our image which is being shown with display() at the end of the
    # function
    print(f"HR {run_ocr(img, hr_left, top, width, height)}")
    print(f"SPO2 {run_ocr(img, spo2_left, top, width, height)}")
    print(f"HRV1 {run_ocr(img, hrv1_left, top, width, height)}")
    print(f"HRV2 {run_ocr(img, hrv2_left, top, width, height)}")

    # Lastly, let's print out the function call signature, so we could easily reuse this later
    print(
        f"draw(hr_left={hr_left}, spo2_left={spo2_left}, hrv1_left={hrv1_left}, hrv2_left={hrv2_left}, top={top}, width={width}, height={height}, do_ocr={do_ocr}, ocr_full_size={ocr_full_size}, ocr_tesseract_options='{ocr_tesseract_options}')"
    )
    display(img)

Ok, there we have it, a simple little user interface to help me explore my data a bit! Now, it's not the prettiest of
code, and I wouldn't write a full application inside of a Jupyter notebook, but people do. The Jupyter widgets allow us
to enhance the development environment a little bit if we're willing to put the effort in, and the `interact` module in
particular makes this pretty easy as we just wrap our function in a decorator object. If you're interested more in this
topic then I highly encourage you to check out the ipywidgets documentation. There are many more sophisticated widgets
available for you to use including layout managers, tabs, buttons, and even video streaming.
